# Caso de Estudio: Predicción de Éxito en Campañas de Telemercadeo con Deep Learning

**Autor:** Luis E. Seijas

**Curso:** Deep Learning para Negocios

---

### 1. Descripción del Problema

En este proyecto, actuarás como Científico de Datos para una institución financiera. El banco necesita optimizar sus campañas de **telemercadeo** para ofrecer depósitos a plazo, un producto financiero clave. Cada llamada tiene un costo, y llamar a clientes que no están interesados no solo es ineficiente, sino que también puede generar una mala experiencia para el cliente.

**El objetivo principal es:**

> Construir un modelo de clasificación basado en redes neuronales profundas que pueda predecir con alta precisión si un cliente suscribirá (`'sí'`) o no (`'no'`) un depósito a plazo después de ser contactado.

Un modelo exitoso permitirá al banco:
*  **Focalizar los esfuerzos:** Concentrar las llamadas en los clientes con mayor probabilidad de conversión.
*  **Reducir costos:** Minimizar el número de llamadas innecesarias.
*  **Aumentar la tasa de éxito:** Mejorar el retorno de inversión (ROI) de las campañas de marketing.

**Es importante considerar:** Los datos son una mezcla de información demográfica, historial bancario y métricas de la campaña. Tu tarea es procesar estos datos, diseñar una arquitectura de red neuronal efectiva y, finalmente, traducir tus resultados en recomendaciones de negocio accionables.

In [ ]:
import tensorflow as tf

print(f"Versión de TensorFlow: {tf.__version__}")

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Esta configuración es una buena práctica para evitar que TensorFlow
        # reserve toda la memoria de la GPU desde el inicio.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"✅ GPU(s) detectada(s): {len(gpus)}. La configuración se aplicó correctamente.")
        print("TensorFlow utilizará la GPU para el entrenamiento.")
    except RuntimeError as e:
        # Este bloque se ejecuta si la GPU ya fue inicializada.
        print(f"⚠️ Error al configurar la GPU: {e}")
        print("La GPU ya estaba inicializada. Si necesitas cambiar la configuración, reinicia el kernel.")
else:
    print("❌ No se detectó ninguna GPU. El entrenamiento se realizará en la CPU.")

#### 1.1. Carga de datos 



In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 1: Configuración del entorno y carga de datos.

En este paso, instalaremos la librería recomendada para acceder al repositorio
de la UCI, importaremos las herramientas necesarias y cargaremos los datos.
"""

# 1. Instalación de la librería de UCI (si no está instalada)
# Descomenta la siguiente línea y ejecútala si es la primera vez que usas esta librería.
# !pip install ucimlrepo

# 2. Librerías para manipulación y análisis de datos
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo 

# 3. Librerías para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Configuraciones para una mejor visualización
sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

# 4. Carga del conjunto de datos usando el ID del repositorio
# ID 222 corresponde al dataset "Bank Marketing"
# Moro, S., Rita, P., & Cortez, P. (2014). Bank Marketing [Dataset]. 
# UCI Machine Learning Repository. https://doi.org/10.24432/C5K306.
bank_marketing = fetch_ucirepo(id=222) 

# Extracción de los datos en DataFrames de pandas
# X contiene las variables predictoras (features)
X = bank_marketing.data.features 
# y contiene la variable objetivo (target)
y = bank_marketing.data.targets 

# Para facilitar el análisis exploratorio, uniremos X e y en un solo DataFrame
df = pd.concat([X, y], axis=1)

print("¡Datos cargados exitosamente usando ucimlrepo!")
print(f"El conjunto de datos tiene {df.shape[0]} filas y {df.shape[1]} columnas.")

# 5. Primera visualización de los datos
# Usamos .head() para mostrar las primeras 5 filas.
print("\nPrimeras 5 filas del conjunto de datos:")
df.head()

### 2. Análisis Exploratorio de Datos (EDA)

Antes de proceder con la construcción de un modelo, es imperativo realizar un Análisis Exploratorio de Datos (EDA). Este proceso consiste en la investigación sistemática del conjunto de datos para identificar patrones, detectar anomalías, validar supuestos y extraer conclusiones iniciales mediante el uso de estadísticas descriptivas y técnicas de visualización.

Los objetivos de esta sección son:
1.  **Analizar la estructura y tipos de datos:** Identificar las variables, sus tipos (numéricas, categóricas) y evaluar la integridad de los datos.
2.  **Examinar la variable objetivo:** Cuantificar la distribución de la variable de salida para identificar posibles sesgos, como el desbalance de clases.
3.  **Visualizar relaciones y distribuciones:** Investigar la relación entre las variables predictoras y la variable objetivo.

In [ ]:
# La librería nos da acceso directo a información valiosa sobre los datos.

# Mostramos la metadata del dataset
print("------ METADATA DEL DATASET ------")
print(bank_marketing.metadata)

# Mostramos la descripción de cada variable
print("\n------ INFORMACIÓN DE LAS VARIABLES ------")
# pd.set_option('display.max_rows', None) # Descomentar para ver todas las variables
print(bank_marketing.variables)

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 2.1: Inspección inicial de la calidad de los datos.

Verificación de tipos de datos por columna y presencia de valores nulos.
"""

# El método .info() proporciona un resumen conciso del DataFrame,
# incluyendo el tipo de dato de cada columna y el conteo de valores no nulos.
print("------ Resumen del DataFrame ------")
df.info()

# Se realiza una comprobación explícita de la suma de valores nulos.
print("\n------ Conteo de Valores Nulos por Columna ------")
print(df.isnull().sum())

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 2.2: Análisis de la variable objetivo 'y'.

Este bloque responde a la Pregunta 1: ¿Cuál es la proporción de clientes que
suscribieron el depósito ('yes') frente a los que no ('no')?
"""

# 1. Conteo de frecuencias absolutas para cada clase.
target_counts = df['y'].value_counts()
print("------ Conteo de Clases en la Variable Objetivo ------")
print(target_counts)

# 2. Cálculo de las frecuencias relativas (porcentajes).
target_percentage = df['y'].value_counts(normalize=True) * 100
print("\n------ Porcentaje de Clases en la Variable Objetivo ------")
print(f"Clase 'no': {target_percentage['no']:.2f}%")
print(f"Clase 'yes': {target_percentage['yes']:.2f}%")

# 3. Visualización de la distribución de la variable objetivo.
plt.figure(figsize=(8, 6))
sns.countplot(x='y', data=df, palette=['#34495e', '#2ecc71'])
plt.title('Distribución de la Variable Objetivo (y)', fontsize=16)
plt.xlabel('Suscripción a Depósito a Plazo', fontsize=12)
plt.ylabel('Frecuencia Absoluta', fontsize=12)
plt.xticks([0, 1], ['No', 'Sí'])

# Anotaciones de porcentaje sobre las barras de la gráfica
for i, percentage in enumerate(target_percentage):
    plt.text(i, target_counts.iloc[i] + 500,
             f'{percentage:.2f}%',
             ha='center', va='center', fontsize=14, color='black')

plt.show()



Los resultados numéricos y la visualización gráfica confirman que el conjunto de datos presenta un **marcado desbalance de clases**.

* **Clase Mayoritaria (`'no'`):** Corresponde al **88.73%** de las instancias.
* **Clase Minoritaria (`'yes'`):** Representa únicamente el **11.27%** de las instancias.

#### **Implicaciones del Desbalance de Clases para el Modelado**

Este desbalance es un factor crítico que debe ser considerado durante el desarrollo y la evaluación del modelo por las siguientes razones:

1.  **Invalidez de la Métrica de Exactitud (`Accuracy`):** Un modelo que clasifique todas las instancias como pertenecientes a la clase mayoritaria (`'no'`) alcanzaría una exactitud del 88.73%. Aunque numéricamente alto, este modelo carecería de toda utilidad práctica, pues su objetivo es, precisamente, identificar a la clase minoritaria. Por lo tanto, la exactitud no es una métrica de evaluación fiable en este contexto.

2.  **Sesgo del Modelo Durante el Entrenamiento:** Los algoritmos de aprendizaje, incluyendo las redes neuronales, tienden a optimizar sus parámetros para minimizar una función de pérdida global. En un escenario desbalanceado, el modelo puede lograr una baja pérdida simplemente al aprender a clasificar correctamente la clase mayoritaria, ignorando los patrones distintivos de la clase minoritaria.

3.  **Requerimiento de Métricas de Evaluación Alternativas:** Para una evaluación de rendimiento robusta, es necesario emplear métricas que sean sensibles al desempeño en la clase minoritaria. Las métricas adecuadas para este problema son:
    * **Precisión (`Precision`):** Mide la proporción de predicciones positivas que fueron correctas. Es fundamental para asegurar la eficiencia de las campañas.
    * **Recall (Sensibilidad o `Recall`):** Mide la proporción de positivos reales que fueron identificados correctamente. Es vital para maximizar la captación de clientes.
    * **Puntuación F1 (`F1-Score`):** La media armónica de Precisión y Recall, que proporciona una medida de rendimiento balanceada.
    * **Área Bajo la Curva ROC (AUC-ROC):** Evalúa la capacidad del modelo para discriminar entre las clases positiva y negativa.

El reconocimiento temprano de este desbalance condiciona la estrategia de modelado y, fundamentalmente, el marco de evaluación del rendimiento del clasificador final.

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 2.3.1: Análisis de la variable numérica 'age'.

Visualizaremos la distribución de la edad para cada una de las clases
de la variable objetivo.
"""

plt.figure(figsize=(12, 7))

# Gráfico de densidad para clientes que dijeron 'no'
sns.kdeplot(df.loc[df['y'] == 'no', 'age'], 
            label='No Suscribió (y=no)', fill=True, color='#34495e')

# Gráfico de densidad para clientes que dijeron 'sí'
sns.kdeplot(df.loc[df['y'] == 'yes', 'age'], 
            label='Sí Suscribió (y=yes)', fill=True, color='#2ecc71')

plt.title('Distribución de Edad por Resultado de la Campaña', fontsize=16)
plt.xlabel('Edad del Cliente', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.legend()
plt.show()

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 2.3.2: Análisis de la variable categórica 'job'.

Se calculará y graficará la tasa de conversión para cada tipo de trabajo.
"""

# Calcular la tasa de conversión por 'job'
# Agrupamos por 'job', calculamos la media de una versión numérica de 'y'
conversion_rate_job = df.groupby('job')['y'].apply(lambda x: (x == 'yes').mean()).sort_values(ascending=False)

# Crear la gráfica
plt.figure(figsize=(14, 8))
ax = sns.barplot(x=conversion_rate_job.index, y=conversion_rate_job.values * 100, palette='viridis')

plt.title('Tasa de Conversión (%) por Tipo de Trabajo', fontsize=16)
plt.xlabel('Tipo de Trabajo', fontsize=12)
plt.ylabel('Tasa de Conversión (%)', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotar etiquetas para mejorar legibilidad

# Añadir anotaciones de porcentaje en las barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 2.3.3: Análisis de la variable temporal 'month'.

Se calculará y graficará la tasa de conversión para cada mes.
"""

# Orden de los meses para una visualización lógica
month_order = ['mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Calcular la tasa de conversión por 'month'
conversion_rate_month = df.groupby('month')['y'].apply(lambda x: (x == 'yes').mean()).reindex(month_order)

# Crear la gráfica
plt.figure(figsize=(12, 7))
ax = sns.barplot(x=conversion_rate_month.index, y=conversion_rate_month.values * 100, palette='plasma')

plt.title('Tasa de Conversión (%) por Mes', fontsize=16)
plt.xlabel('Mes del Último Contacto', fontsize=12)
plt.ylabel('Tasa de Conversión (%)', fontsize=12)

# Añadir anotaciones de porcentaje
for p in ax.patches:
    if p.get_height() > 0: # Solo anotar si hay valor
        ax.annotate(f'{p.get_height():.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', fontsize=11, color='black', xytext=(0, 5),
                    textcoords='offset points')

plt.show()

## 3. Preprocesamiento de Datos para el Modelo de Deep Learning

Las redes neuronales, en su forma más común, operan exclusivamente con datos numéricos. Nuestro conjunto de datos, sin embargo, contiene una mezcla de variables numéricas y categóricas. Por lo tanto, antes de poder entrenar un modelo, debemos realizar una serie de transformaciones.

El preprocesamiento de datos para este proyecto se centrará en tres tareas principales:

1.  **Codificación de la Variable Objetivo:** Convertir la variable `y` ('yes'/'no') a un formato numérico (1/0).
2.  **Codificación de Variables Categóricas:** Transformar las variables de texto (como `job` o `marital`) en una representación numérica que el modelo pueda interpretar.
3.  **Escalado de Características Numéricas:** Estandarizar las variables numéricas (como `age` o `duration`) para que tengan una media de 0 y una desviación estándar de 1. Esto es crucial para la correcta convergencia de los algoritmos de optimización como el descenso de gradiente.


In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 3.1: Pipeline de preprocesamiento de datos.

Se aplicarán técnicas de codificación y escalado para preparar los datos
para el entrenamiento de la red neuronal.
"""

# 1. Importar las herramientas necesarias de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 2. Separar las características (X) y la variable objetivo (y) del DataFrame original
X = df.drop('y', axis=1)
y = df['y']

# 3. Codificar la variable objetivo 'y' a formato numérico (0 y 1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# 'no' se codifica como 0, 'yes' como 1.
print(f"Clases de la variable objetivo: {label_encoder.classes_}")
print(f"Primeras 10 etiquetas codificadas: {y_encoded[:10]}")


# 4. Identificar las columnas numéricas y categóricas
# Considere en este punto que datos deben excluirse 
# A modo de ejemplo mantendremos todas características, 
# pero la seleccion es una discusión importante.
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

print(f"\nCaracterísticas numéricas ({len(numeric_features)}): {numeric_features}")
print(f"Características categóricas ({len(categorical_features)}): {categorical_features}")


# 5. Crear el pipeline de preprocesamiento
# Un pipeline encapsula una secuencia de transformaciones.

# Pipeline para características numéricas: solo se necesita escalado.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline para características categóricas: se necesita codificación One-Hot.
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
    # handle_unknown='ignore' evita errores si aparecen nuevas categorías en test.
    # drop='first' evita multicolinealidad al eliminar una categoría por variable.
])

# 6. Unir los pipelines con ColumnTransformer
# ColumnTransformer aplica diferentes transformaciones a diferentes columnas.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Mantiene columnas no especificadas (si las hubiera)
)

# 7. Dividir los datos en conjuntos de entrenamiento y prueba
# Se reserva un 20% de los datos para la evaluación final del modelo.
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    # stratify=y_encoded asegura que la proporción de clases sea la misma
    # en los conjuntos de entrenamiento y prueba, lo cual es vital.
)

# 8. Aplicar el preprocesamiento
# .fit_transform() en los datos de entrenamiento para aprender los parámetros
# de escalado y codificación.
X_train_processed = preprocessor.fit_transform(X_train)

# .transform() en los datos de prueba usando los parámetros aprendidos del
# conjunto de entrenamiento. Esto evita la fuga de datos (data leakage).
X_test_processed = preprocessor.transform(X_test)


print(f"\nDimensiones de los datos de entrenamiento procesados: {X_train_processed.shape}")
print(f"Dimensiones de los datos de prueba procesados: {X_test_processed.shape}")



La preparación de los datos para el modelo de red neuronal se realizó siguiendo una estrategia estructurada, cuyas decisiones se justifican a continuación.

#### **Manejo de Variables Categóricas**

Para las variables categóricas (`job`, `marital`, `education`, etc.), se seleccionó la técnica de **Codificación One-Hot (`OneHotEncoder`)**.

* **Justificación:** Esta técnica crea una nueva columna binaria (0 o 1) para cada categoría dentro de una variable. Se prefiere sobre otras técnicas como la Codificación de Etiquetas (`LabelEncoder`) para variables nominales (donde las categorías no tienen un orden intrínseco) por una razón fundamental:
    * **Evita la Creación de un Orden Artificial:** `LabelEncoder` asignaría a cada categoría un entero (e.g., `student`=1, `retired`=2, `services`=3). El modelo podría interpretar erróneamente que existe una relación ordinal (e.g., que `services` > `retired`), lo cual es incorrecto y podría introducir un sesgo no deseado. `OneHotEncoder` trata cada categoría como una entidad independiente, eliminando este riesgo.
    * Se utilizó el parámetro `drop='first'` para eliminar una de las columnas generadas por cada variable, evitando así la multicolinealidad perfecta, lo cual es una buena práctica en modelado.

#### **Manejo de Variables Numéricas**

Para las variables numéricas (`age`, `campaign`, `euribor3m`, etc.), se aplicó la **Estandarización (`StandardScaler`)**.

* **Justificación:** La estandarización transforma los datos para que tengan una **media de 0 y una desviación estándar de 1**. Esta técnica es crucial para el entrenamiento de redes neuronales por dos motivos principales:
    * **Convergencia del Optimizador:** Los algoritmos de optimización basados en gradiente, como *Adam* o *SGD*, convergen mucho más rápido cuando las características se encuentran en una escala similar. Si una característica tiene un rango de valores mucho mayor que otras (e.g., `pdays` vs. `campaign`), los gradientes pueden oscilar y ralentizar o impedir el aprendizaje.
    * **Igualdad de Contribución Inicial:** La estandarización asegura que todas las características tengan el mismo "peso" inicial en el cálculo de la función de pérdida, permitiendo que el modelo aprenda su importancia real a través de los pesos sinápticos de manera más efectiva.

#### **División de Datos Estratificada**

Finalmente, el conjunto de datos se dividió en un 80% para entrenamiento y un 20% para prueba, utilizando una **división estratificada (`stratify=y_encoded`)**.

* **Justificación:** Dado el severo desbalance de clases (89% vs. 11%), una división aleatoria simple podría resultar en una proporción de clases significativamente diferente entre los conjuntos de entrenamiento y prueba. La estratificación garantiza que esta proporción se mantenga constante en ambas divisiones, lo que permite una evaluación del modelo mucho más fiable y representativa del problema original.

#### El Impacto de la Fuga de Datos con la Variable 'duration'
* **Contexto**: La fuga de datos (data leakage) ocurre cuando se utiliza información en el entrenamiento del modelo que no estaría disponible en un escenario de predicción real. La variable duration (duración de la llamada) es un ejemplo clásico: una llamada larga a menudo se correlaciona con el interés del cliente, pero no conocemos su duración antes de realizarla.

    * Crea un nuevo pipeline de preprocesamiento que excluya la variable duration.

    * Entrena el mismo modelo base con este nuevo conjunto de datos.

    * Compara la métrica AUC del modelo entrenado con duration versus el modelo entrenado sin duration.

```
# --- Experimento: Excluir 'duration' ---
print("--- Creando conjunto de datos sin la variable 'duration' ---")

# Excluir 'duration' de las características numéricas
numeric_features_no_duration = [feat for feat in numeric_features if feat != 'duration']

# Crear un nuevo preprocesador
preprocessor_no_duration = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features_no_duration),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# Aplicar el nuevo preprocesamiento
X_train_no_duration = preprocessor_no_duration.fit_transform(X_train)
X_test_no_duration = preprocessor_no_duration.transform(X_test)

print(f"Dimensiones de los datos sin 'duration': {X_train_no_duration.shape}")
```

## 4. Diseño y Entrenamiento del Modelo de Deep Learning

Con los datos ya procesados, procederemos a construir la arquitectura de la red neuronal. El diseño de la arquitectura es un paso fundamental que implica definir el número de capas, la cantidad de neuronas en cada capa, las funciones de activación y las técnicas para prevenir el sobreajuste.

Posteriormente, compilaremos el modelo especificando el optimizador y la función de pérdida, y lo entrenaremos con nuestros datos. Esta sección responderá a las **Preguntas 4 y 5**.

#### **Tarea: Definir la Arquitectura de su Modelo**

Utilizando la función `build_classifier` definida más abajo, su tarea es experimentar y definir la arquitectura de la red neuronal. Modifique los valores en la siguiente celda de código para explorar cómo los cambios afectan el rendimiento del modelo.

**Parámetros a experimentar:**
* `HIDDEN_LAYERS`: Pruebe con más o menos capas, y con diferente número de neuronas (e.g., `(128, 64, 32)`, `(32,)`, `(100, 50)`).
* `DROPOUT_RATE`: Varíe la tasa de dropout (e.g., `0.2`, `0.5`) o desactívelo (`0.0`).
* `LEARNING_RATE`: Pruebe con tasas de aprendizaje más altas o más bajas (e.g., `0.01`, `0.0001`).

Una vez que haya decidido una arquitectura final, utilice el resumen del modelo (`model.summary()`) y la justificación de la celda siguiente para responder a la **Pregunta 4**.

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 4.1: Función reutilizable para construir el modelo clasificador.

Se define una función que encapsula la lógica de construcción y compilación
de la red neuronal, permitiendo modificar su arquitectura fácilmente a través
de parámetros.
"""

# 1. Importar las clases y funciones necesarias de TensorFlow y Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_classifier(n_features, hidden_layers=(64, 32),
                     activation='relu', dropout_rate=0.3,
                     learning_rate=0.001):
    """Construye y compila un modelo de red neuronal secuencial de Keras.

    Esta función crea un clasificador binario con una arquitectura flexible.
    La estructura consiste en una capa de entrada, un número variable de capas
    ocultas densas (con Dropout opcional) y una capa de salida sigmoide.

    Args:
        n_features (int):
            El número de características de entrada del modelo. Corresponde
            a la dimensionalidad de los datos de entrada (shape[1]).
        hidden_layers (tuple of int, optional):
            Una tupla donde cada entero representa el número de neuronas en una
            capa oculta. El número de elementos en la tupla determina el
            número de capas ocultas.
            Defaults to (64, 32).
        activation (str, optional):
            La función de activación a utilizar en las capas ocultas.
            Defaults to 'relu'.
        dropout_rate (float, optional):
            La tasa de Dropout a aplicar después de cada capa oculta. Debe ser
            un valor entre 0.0 y 1.0. Si es 0, no se aplicará Dropout.
            Defaults to 0.3.
        learning_rate (float, optional):
            La tasa de aprendizaje para el optimizador Adam.
            Defaults to 0.001.

    Returns:
        tensorflow.keras.models.Sequential:
            Un modelo de Keras compilado y listo para ser entrenado.
    """
    # Inicializar el modelo secuencial, que es un stack lineal de capas.
    model = Sequential(name="Bank_Marketing_Classifier")

    # --- Capa de Entrada ---
    # La primera capa Dense también define la capa de entrada a través del
    # argumento `input_shape`. Se añade automáticamente.
    model.add(Dense(
        units=hidden_layers[0],
        activation=activation,
        input_shape=(n_features,),
        name=f"Capa_Oculta_1_{hidden_layers[0]}_neuronas"
    ))
    
    # Aplicar Dropout si la tasa es mayor que cero.
    if dropout_rate > 0:
        model.add(Dropout(dropout_rate, name="Dropout_1"))

    # --- Capas Ocultas Adicionales ---
    # Iterar sobre el resto de las capas definidas en la tupla `hidden_layers`.
    # Se empieza desde el segundo elemento (índice 1).
    for i, neurons in enumerate(hidden_layers[1:], start=2):
        model.add(Dense(
            units=neurons,
            activation=activation,
            name=f"Capa_Oculta_{i}_{neurons}_neuronas"
        ))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate, name=f"Dropout_{i}"))

    # --- Capa de Salida ---
    # Para clasificación binaria, se usa una única neurona con activación sigmoide.
    # La salida será una probabilidad (un valor entre 0 y 1).
    # Esto es una sugerencia pero puede modificarlo y probar otras alternativas.
    model.add(Dense(1, activation='sigmoid', name="Capa_Salida_Sigmoide"))

    # --- Compilación del Modelo ---
    # Crear una instancia del optimizador Adam con la tasa de aprendizaje definida.
    optimizer = Adam(learning_rate=learning_rate)
    
    # Configurar el proceso de aprendizaje del modelo.
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy', # Función de pérdida para clasificación binaria.
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )

    return model

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 4.2: Configuración de hiperparámetros y construcción del modelo.

Los estudiantes deben modificar estas variables para definir su arquitectura.
"""

# --- Panel de Control de la Arquitectura ---
# Modifique estos valores para experimentar.

# Tupla con las neuronas de cada capa oculta.
# Ejemplo: (64, 32) -> Dos capas ocultas, la primera con 64 neuronas, la segunda con 32.
HIDDEN_LAYERS = (32,)

# Tasa de Dropout (0.0 para desactivar).
DROPOUT_RATE = 0.0

# Tasa de aprendizaje para el optimizador Adam.
LEARNING_RATE = 0.1
# ---------------------------------------------


# Obtener el número de características del conjunto de datos procesado.
n_features = X_train_processed.shape[1]

# Llamar a la función para construir el modelo con los parámetros definidos.
model = build_classifier(
    n_features,
    hidden_layers=HIDDEN_LAYERS,
    dropout_rate=DROPOUT_RATE,
    learning_rate=LEARNING_RATE
)

# Imprimir el resumen de la arquitectura del modelo final.
model.summary()

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 4.3: Entrenamiento del modelo y aplicación de técnicas contra el sobreajuste.

Se entrena el modelo compilado utilizando los datos de entrenamiento y se
implementa EarlyStopping para finalizar el proceso de forma óptima.
"""

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
import numpy as np

# Calcula los pesos de clase para que sean inversamente proporcionales a su frecuencia.
# La clase minoritaria ('yes', codificada como 1) recibirá un peso mucho mayor.
weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

# El resultado será algo como: array([ 0.56, 4.43])
# Significa que un error en la clase '1' ('yes') penalizará ~8 veces más
# que un error en la clase '0' ('no').
class_weights = {i : weights[i] for i,_ in enumerate(weights)}

print("Pesos de Clase Calculados:")
print(class_weights)

# 1. Definir el callback de EarlyStopping para prevenir el sobreajuste.
# Esta técnica monitorea una métrica de interés (la pérdida en el conjunto de
# validación, 'val_loss') y detiene el entrenamiento si no se observa una
# mejora tras un número definido de épocas ('patience').

early_stopping = EarlyStopping(
    monitor='val_loss', # Métrica a monitorear.
    patience=10,        # Nº de épocas a esperar sin mejora antes de parar.
    verbose=1,          # Informar en consola cuando el entrenamiento se detiene.
    mode='min',         # La monitorización se detiene cuando la métrica deja de disminuir.
    restore_best_weights=True # Restaura los pesos del modelo de la mejor época.
)

# 2. Entrenar el modelo con el método .fit().
# Se proporciona el conjunto de datos de entrenamiento (X_train_processed, y_train).
# El entrenamiento se ejecutará por un máximo de 100 épocas, pero EarlyStopping
# probablemente lo detendrá antes.

print("\n------ Iniciando Entrenamiento del Modelo ------")
history = model.fit(
    X_train_processed,
    y_train,
    epochs=100,             # Número máximo de épocas.
    batch_size=512,          # Número de muestras por actualización de gradiente.
    validation_split=0.2,   # Porcentaje de datos de entrenamiento a usar para validación.
    callbacks=[early_stopping], # Lista de callbacks a aplicar durante el entrenamiento.
    verbose=1               # Muestra una barra de progreso.
)
print("------ Entrenamiento Finalizado ------")

In [ ]:
# -*- coding: utf-8 -*-
"""
Visualización del rendimiento durante el entrenamiento.

Graficar las curvas de aprendizaje (pérdida y métricas) para los conjuntos de
entrenamiento y validación es esencial para diagnosticar el sobreajuste y
evaluar el proceso de aprendizaje del modelo.
"""
# El objeto 'history' devuelto por model.fit() contiene un diccionario
# con los valores de pérdida y métricas de cada época.
# Lo convertimos a un DataFrame de pandas para facilitar la manipulación.
history_df = pd.DataFrame(history.history)

# Creamos una figura con dos subplots, uno al lado del otro.
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# --- Gráfico de la Función de Pérdida (Loss) ---
# Compara cómo evolucionó el error en los datos de entrenamiento vs. validación.
axes[0].plot(history_df['loss'], label='Pérdida de Entrenamiento', color='#3498db', lw=2)
axes[0].plot(history_df['val_loss'], label='Pérdida de Validación', color='#e74c3c', lw=2, linestyle='--')
axes[0].set_title('Curvas de Aprendizaje: Pérdida', fontsize=16)
axes[0].set_xlabel('Épocas', fontsize=12)
axes[0].set_ylabel('Binary Cross-Entropy', fontsize=12)
axes[0].legend()
axes[0].grid(True)

# --- Gráfico de la Métrica de Rendimiento (AUC) ---
# Compara el rendimiento (AUC) en los datos de entrenamiento vs. validación.
# Usamos AUC en lugar de Accuracy por ser más robusto al desbalance.
axes[1].plot(history_df['auc'], label='AUC de Entrenamiento', color='#3498db', lw=2)
axes[1].plot(history_df['val_auc'], label='AUC de Validación', color='#e74c3c', lw=2, linestyle='--')
axes[1].set_title('Curvas de Aprendizaje: AUC', fontsize=16)
axes[1].set_xlabel('Épocas', fontsize=12)
axes[1].set_ylabel('AUC', fontsize=12)
axes[1].legend()
axes[1].grid(True)

# Ajusta el layout para evitar solapamientos y muestra la figura.
plt.tight_layout()
plt.show()

## 5. Evaluación del Rendimiento del Modelo

Ha llegado el momento de la verdad. Hasta ahora, hemos entrenado el modelo y monitoreado su rendimiento utilizando un conjunto de validación derivado de los datos de entrenamiento. Ahora, evaluaremos su capacidad de generalización final utilizando el **conjunto de prueba (`test set`)**, que el modelo no ha visto en ninguna etapa anterior.

Esta evaluación nos permitirá obtener una estimación imparcial de cómo se comportaría el modelo en un entorno de producción con datos nuevos. Nos centraremos en las métricas de clasificación clave para abordar la **Pregunta 6**.

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 5.1: Evaluación del modelo en el conjunto de prueba.

Se calculan las métricas de rendimiento y la matriz de confusión.
"""

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns

# 1. Evaluar el modelo usando el método .evaluate() de Keras
# Este método devuelve la pérdida (loss) y las métricas que definimos
# al compilar el modelo (accuracy, auc).
print("------ Evaluación General de Keras ------")
loss, accuracy, auc_score = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Exactitud (Accuracy) en el conjunto de prueba: {accuracy:.4f}")
print(f"Área Bajo la Curva ROC (AUC) en el conjunto de prueba: {auc_score:.4f}")


# 2. Realizar predicciones de probabilidad
# El método .predict() devuelve la salida de la capa sigmoide.
y_pred_proba = model.predict(X_test_processed).flatten()

# 3. Convertir probabilidades a clases binarias (0 o 1)
# Se utiliza un umbral estándar de 0.5.
y_pred_class = (y_pred_proba > 0.5).astype(int)


# 4. Generar el reporte de clasificación de scikit-learn
# Este reporte incluye Precisión, Recall y F1-Score.
print("\n------ Reporte de Clasificación Detallado ------")
print(classification_report(y_test, y_pred_class, target_names=['No', 'Sí']))


# 5. Generar y visualizar la Matriz de Confusión
print("\n------ Matriz de Confusión ------")
cm = confusion_matrix(y_test, y_pred_class)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Predicho No', 'Predicho Sí'],
            yticklabels=['Real No', 'Real Sí'])
plt.title('Matriz de Confusión', fontsize=16)
plt.ylabel('Etiqueta Real', fontsize=12)
plt.xlabel('Etiqueta Predicha', fontsize=12)
plt.show()

In [ ]:
# -*- coding: utf-8 -*-
"""
Paso 5.2: Visualización de la Curva PR.

Esta curva muestra el rendimiento de un modelo de clasificación en todos
los umbrales de clasificación.
"""

from sklearn.metrics import precision_recall_curve

precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)

plt.figure(figsize=(10, 7))
plt.plot(recall, precision, color='purple', lw=2)
plt.xlabel('Recall (Sensibilidad)')
plt.ylabel('Precisión')
plt.title('Curva de Precisión-Recall')
plt.grid(True)
plt.show()